# Mécanismes d'attention et transformateurs

Un inconvénient majeur des réseaux récurrents est que tous les mots d'une séquence ont le même impact sur le résultat. Cela entraîne des performances sous-optimales avec les modèles standard encodeur-décodeur LSTM pour les tâches de séquence à séquence, telles que la reconnaissance d'entités nommées et la traduction automatique. En réalité, certains mots spécifiques de la séquence d'entrée ont souvent plus d'impact sur les sorties séquentielles que d'autres.

Prenons un modèle de séquence à séquence, comme la traduction automatique. Il est implémenté par deux réseaux récurrents, où un réseau (**encodeur**) compresse la séquence d'entrée dans un état caché, et un autre (**décodeur**) déploie cet état caché pour produire le résultat traduit. Le problème avec cette approche est que l'état final du réseau a du mal à se souvenir du début de la phrase, ce qui entraîne une mauvaise qualité du modèle pour les phrases longues.

Les **mécanismes d'attention** offrent un moyen de pondérer l'impact contextuel de chaque vecteur d'entrée sur chaque prédiction de sortie du RNN. Cela est réalisé en créant des raccourcis entre les états intermédiaires du RNN d'entrée et du RNN de sortie. Ainsi, lors de la génération du symbole de sortie $y_t$, nous prenons en compte tous les états cachés d'entrée $h_i$, avec différents coefficients de pondération $\alpha_{t,i}$.

![Image montrant un modèle encodeur/décodeur avec une couche d'attention additive](../../../../../lessons/5-NLP/18-Transformers/images/encoder-decoder-attention.png)
*Le modèle encodeur-décodeur avec mécanisme d'attention additive dans [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf), cité de [cet article de blog](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html)*

La matrice d'attention $\{\alpha_{i,j}\}$ représente le degré auquel certains mots d'entrée influencent la génération d'un mot donné dans la séquence de sortie. Voici un exemple de cette matrice :

![Image montrant un alignement trouvé par RNNsearch-50, tirée de Bahdanau - arviz.org](../../../../../lessons/5-NLP/18-Transformers/images/bahdanau-fig3.png)

*Figure tirée de [Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) (Fig.3)*

Les mécanismes d'attention sont responsables de la plupart des avancées actuelles ou proches de l'état de l'art en traitement du langage naturel. Cependant, l'ajout d'attention augmente considérablement le nombre de paramètres du modèle, ce qui a entraîné des problèmes de mise à l'échelle avec les RNN. Une contrainte clé pour la mise à l'échelle des RNN est que la nature récurrente des modèles rend difficile le traitement par lots et la parallélisation de l'entraînement. Dans un RNN, chaque élément d'une séquence doit être traité dans un ordre séquentiel, ce qui signifie qu'il ne peut pas être facilement parallélisé.

L'adoption des mécanismes d'attention combinée à cette contrainte a conduit à la création des modèles transformateurs, désormais à l'état de l'art, que nous connaissons et utilisons aujourd'hui, de BERT à OpenGPT3.

## Modèles transformateurs

Au lieu de transmettre le contexte de chaque prédiction précédente à l'étape d'évaluation suivante, les **modèles transformateurs** utilisent des **encodages positionnels** et **l'attention** pour capturer le contexte d'une entrée donnée dans une fenêtre de texte fournie. L'image ci-dessous montre comment les encodages positionnels avec attention peuvent capturer le contexte dans une fenêtre donnée.

![GIF animé montrant comment les évaluations sont effectuées dans les modèles transformateurs.](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif)

Étant donné que chaque position d'entrée est mappée indépendamment à chaque position de sortie, les transformateurs peuvent mieux paralléliser que les RNN, ce qui permet des modèles de langage beaucoup plus grands et plus expressifs. Chaque tête d'attention peut être utilisée pour apprendre différentes relations entre les mots, ce qui améliore les tâches de traitement du langage naturel en aval.

## Construire un modèle transformateur simple

Keras ne contient pas de couche transformateur intégrée, mais nous pouvons en construire une nous-mêmes. Comme précédemment, nous nous concentrerons sur la classification de texte du jeu de données AG News, mais il convient de mentionner que les modèles transformateurs donnent les meilleurs résultats pour des tâches NLP plus complexes.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

Les nouvelles couches dans Keras doivent hériter de la classe `Layer` et implémenter la méthode `call`. Commençons par la couche **Positional Embedding**. Nous utiliserons [du code provenant de la documentation officielle de Keras](https://keras.io/examples/nlp/text_classification_with_transformer/). Nous supposerons que nous remplissons toutes les séquences d'entrée à une longueur `maxlen`.


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

Cette couche se compose de deux couches `Embedding` : une pour l'intégration des tokens (comme nous l'avons déjà abordé) et une pour les positions des tokens. Les positions des tokens sont générées comme une séquence de nombres naturels allant de 0 à `maxlen` à l'aide de `tf.range`, puis passées à travers la couche d'intégration. Les deux vecteurs d'intégration résultants sont ensuite additionnés, produisant une représentation intégrée positionnelle de l'entrée de forme `maxlen`$\times$`embed_dim`.

Passons maintenant à l'implémentation du bloc transformateur. Il prendra en entrée le résultat de la couche d'intégration définie précédemment :


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

Maintenant, nous sommes prêts à définir le modèle complet du transformeur :


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## Modèles Transformers BERT

**BERT** (Bidirectional Encoder Representations from Transformers) est un réseau de transformateurs multi-couches très large, avec 12 couches pour *BERT-base* et 24 pour *BERT-large*. Le modèle est d'abord pré-entraîné sur un vaste corpus de données textuelles (WikiPedia + livres) en utilisant un apprentissage non supervisé (prédiction des mots masqués dans une phrase). Pendant cette phase de pré-entraînement, le modèle acquiert un niveau significatif de compréhension du langage, qui peut ensuite être exploité avec d'autres ensembles de données grâce à un ajustement fin. Ce processus est appelé **apprentissage par transfert**.

![image provenant de http://jalammar.github.io/illustrated-bert/](../../../../../lessons/5-NLP/18-Transformers/images/jalammarBERT-language-modeling-masked-lm.png)

Il existe de nombreuses variantes des architectures Transformer, notamment BERT, DistilBERT, BigBird, OpenGPT3 et bien d'autres, qui peuvent être ajustées.

Voyons comment nous pouvons utiliser un modèle BERT pré-entraîné pour résoudre notre problème classique de classification de séquences. Nous allons emprunter l'idée et une partie du code de la [documentation officielle](https://www.tensorflow.org/text/tutorials/classify_text_with_bert).

Pour charger des modèles pré-entraînés, nous utiliserons **Tensorflow hub**. Tout d'abord, chargeons le vectoriseur spécifique à BERT :


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

Il est important d'utiliser le même vectoriseur que celui avec lequel le réseau original a été entraîné. De plus, le vectoriseur BERT renvoie trois composants :
* `input_word_ids`, qui est une séquence de numéros de tokens pour la phrase d'entrée
* `input_mask`, qui indique quelle partie de la séquence contient l'entrée réelle et laquelle est du remplissage. Cela est similaire au masque produit par la couche `Masking`
* `input_type_ids` est utilisé pour les tâches de modélisation de langage et permet de spécifier deux phrases d'entrée dans une seule séquence.

Ensuite, nous pouvons instancier l'extracteur de caractéristiques BERT :


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


Ainsi, la couche BERT retourne plusieurs résultats utiles :
* `pooled_output` est le résultat de la moyenne de tous les tokens dans la séquence. Vous pouvez le considérer comme une représentation sémantique intelligente de tout le réseau. Cela équivaut à la sortie de la couche `GlobalAveragePooling1D` dans notre modèle précédent.
* `sequence_output` est la sortie de la dernière couche du transformeur (correspond à la sortie de `TransformerBlock` dans notre modèle ci-dessus).
* `encoder_outputs` sont les sorties de toutes les couches du transformeur. Étant donné que nous avons chargé un modèle BERT à 4 couches (comme vous pouvez probablement le deviner d'après le nom, qui contient `4_H`), il possède 4 tenseurs. Le dernier est identique à `sequence_output`.

Nous allons maintenant définir le modèle de classification de bout en bout. Nous utiliserons une *définition fonctionnelle de modèle*, où nous définissons l'entrée du modèle, puis fournissons une série d'expressions pour calculer sa sortie. Nous rendrons également les poids du modèle BERT non entraînables et entraînerons uniquement le classificateur final :


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


Bien que le nombre de paramètres entraînables soit faible, le processus est assez lent, car l'extracteur de caractéristiques BERT est très gourmand en calcul. Il semble que nous n'ayons pas réussi à atteindre une précision raisonnable, soit par manque d'entraînement, soit par insuffisance des paramètres du modèle.

Essayons de déverrouiller les poids de BERT et de l'entraîner également. Cela nécessite un taux d'apprentissage très faible, ainsi qu'une stratégie d'entraînement plus prudente avec un **warmup**, en utilisant l'optimiseur **AdamW**. Nous utiliserons le package `tf-models-official` pour créer l'optimiseur :


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

Comme vous pouvez le constater, l'entraînement progresse assez lentement - mais vous pourriez vouloir expérimenter et entraîner le modèle pendant quelques époques (5-10) pour voir si vous pouvez obtenir un meilleur résultat par rapport aux approches que nous avons utilisées auparavant.

## Bibliothèque Huggingface Transformers

Une autre méthode très courante (et un peu plus simple) pour utiliser les modèles Transformer est le [package HuggingFace](https://github.com/huggingface/), qui fournit des blocs de construction simples pour différentes tâches de PNL. Il est disponible à la fois pour Tensorflow et PyTorch, un autre framework de réseaux neuronaux très populaire.

> **Note** : Si vous n'êtes pas intéressé par le fonctionnement de la bibliothèque Transformers - vous pouvez passer directement à la fin de ce notebook, car vous ne verrez rien de fondamentalement différent de ce que nous avons fait précédemment. Nous allons répéter les mêmes étapes d'entraînement du modèle BERT en utilisant une bibliothèque différente et un modèle sensiblement plus grand. Par conséquent, le processus implique un entraînement assez long, donc vous pourriez simplement vouloir parcourir le code.

Voyons comment notre problème peut être résolu en utilisant [Huggingface Transformers](http://huggingface.co).


La première chose à faire est de choisir le modèle que nous allons utiliser. En plus de certains modèles intégrés, Huggingface propose un [répertoire de modèles en ligne](https://huggingface.co/models), où vous pouvez trouver de nombreux modèles pré-entraînés par la communauté. Tous ces modèles peuvent être chargés et utilisés simplement en fournissant un nom de modèle. Tous les fichiers binaires nécessaires au modèle seront automatiquement téléchargés.

Parfois, vous devrez charger vos propres modèles. Dans ce cas, vous pouvez spécifier le répertoire contenant tous les fichiers pertinents, y compris les paramètres pour le tokenizer, le fichier `config.json` avec les paramètres du modèle, les poids binaires, etc.

À partir du nom du modèle, nous pouvons instancier à la fois le modèle et le tokenizer. Commençons par un tokenizer :


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

L'objet `tokenizer` contient la fonction `encode` qui peut être utilisée directement pour encoder du texte :


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

Nous pouvons également utiliser le tokenizer pour encoder une séquence d'une manière adaptée à son passage au modèle, c'est-à-dire en incluant les champs `token_ids`, `input_mask`, etc. Nous pouvons également spécifier que nous voulons des tenseurs Tensorflow en fournissant l'argument `return_tensors='tf'` :


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

Dans notre cas, nous utiliserons un modèle BERT pré-entraîné appelé `bert-base-uncased`. *Uncased* signifie que le modèle est insensible à la casse.

Lors de l'entraînement du modèle, nous devons fournir une séquence tokenisée en entrée, et pour cela, nous concevrons un pipeline de traitement des données. Étant donné que `tokenizer.encode` est une fonction Python, nous utiliserons la même approche que dans l'unité précédente en l'appelant avec `py_function` :


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

Maintenant, nous pouvons charger le modèle réel en utilisant le package `BertForSequenceClassification`. Cela garantit que notre modèle dispose déjà de l'architecture requise pour la classification, y compris le classificateur final. Vous verrez un message d'avertissement indiquant que les poids du classificateur final ne sont pas initialisés, et que le modèle nécessiterait un pré-entraînement - c'est tout à fait normal, car c'est exactement ce que nous allons faire !


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


Comme vous pouvez le voir dans `summary()`, le modèle contient presque 110 millions de paramètres ! Présumément, si nous voulons une tâche de classification simple sur un ensemble de données relativement petit, nous ne voulons pas entraîner la couche de base de BERT :


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


Nous sommes maintenant prêts à commencer l'entraînement !

> **Note** : L'entraînement d'un modèle BERT à grande échelle peut être très long ! C'est pourquoi nous allons seulement l'entraîner sur les 32 premiers lots. Cela sert simplement à montrer comment l'entraînement du modèle est configuré. Si vous souhaitez essayer un entraînement à grande échelle, il vous suffit de supprimer les paramètres `steps_per_epoch` et `validation_steps`, et de vous préparer à patienter !


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


Si vous augmentez le nombre d'itérations, patientez suffisamment longtemps et entraînez pendant plusieurs époques, vous pouvez vous attendre à ce que la classification avec BERT offre la meilleure précision ! Cela s'explique par le fait que BERT comprend déjà très bien la structure de la langue, et qu'il suffit simplement d'ajuster le classificateur final. Cependant, comme BERT est un modèle volumineux, tout le processus d'entraînement prend beaucoup de temps et nécessite une puissance de calcul conséquente ! (GPU, et de préférence plusieurs).

> **Note :** Dans notre exemple, nous utilisons l'un des plus petits modèles BERT pré-entraînés. Il existe des modèles plus grands qui sont susceptibles de produire de meilleurs résultats.


## À retenir

Dans cette unité, nous avons exploré des architectures de modèles très récentes basées sur les **transformers**. Nous les avons appliquées à notre tâche de classification de texte, mais de la même manière, les modèles BERT peuvent être utilisés pour l'extraction d'entités, le questionnement automatique et d'autres tâches de NLP.

Les modèles basés sur les transformers représentent l'état de l'art actuel en NLP, et dans la plupart des cas, ils devraient être la première solution à expérimenter lorsque vous implémentez des solutions NLP personnalisées. Cependant, comprendre les principes fondamentaux des réseaux neuronaux récurrents abordés dans ce module est extrêmement important si vous souhaitez concevoir des modèles neuronaux avancés.



---

**Avertissement** :  
Ce document a été traduit à l'aide du service de traduction automatique [Co-op Translator](https://github.com/Azure/co-op-translator). Bien que nous nous efforcions d'assurer l'exactitude, veuillez noter que les traductions automatisées peuvent contenir des erreurs ou des inexactitudes. Le document original dans sa langue d'origine doit être considéré comme la source faisant autorité. Pour des informations critiques, il est recommandé de recourir à une traduction professionnelle réalisée par un humain. Nous déclinons toute responsabilité en cas de malentendus ou d'interprétations erronées résultant de l'utilisation de cette traduction.
